In [ ]:
#Primer paso que se realizo fue importar todas las librerias que vamos a utilizar
import pandas as pd
import json
import re
from fastapi import FastAPI
from datetime import datetime

In [ ]:
# Crear una instancia de FastAPI
app = FastAPI()
#Formula para Director
@app.get("/director/{director}")
def get_director(director: str):
    df1 = df[df["director"].notna() &df["director"].str.contains(director)]
    df1["ganancia"] = df1["revenue"] - df1["budget"]
    retorno_director = df1["return"].sum()
    resp = df1[["title", "release_year", "return", "budget", "ganancia"]]

    return [director, retorno_director, resp]

In [ ]:
## Formula para VOTOS
@app.get('/votos_titulo')
def votos_titulo(titulo_de_la_filmacion):
    pelicula = df[df["title"] == titulo_de_la_filmacion]
    
    if pelicula.empty:
        return "No se encontró la filmación en el dataset."
    
    cantidad_votos = pelicula["vote_count"].iloc[0]
    
    if cantidad_votos < 2000:
        return "La filmación no cumple con la cantidad mínima de valoraciones."
    
    promedio_votos = pelicula["vote_average"].iloc[0]
    mensaje = f"La película {titulo_de_la_filmacion} fue estrenada en el año {pelicula['release_year'].iloc[0]}. Cuenta con un total de {cantidad_votos} valoraciones, con un promedio de {promedio_votos}."
    
    return mensaje

In [ ]:
## Formula para Actor
@app.get('/get_actor')
def get_actor(nombre_actor):
    actor_df = df[df["actors"].str.contains(nombre_actor, case=False, na=False)]
    actor_df["retorno"] = actor_df["revenue"] - actor_df["budget"]
    
    cantidad_peliculas = len(actor_df)
    retorno_total = actor_df["retorno"].sum()
    promedio_retorno = actor_df["retorno"].mean()
    
    mensaje = f"El actor {nombre_actor} ha participado en {cantidad_peliculas} filmaciones. Ha conseguido un retorno total de {retorno_total} con un promedio de {promedio_retorno} por filmación."
    
    return mensaje

In [ ]:
# Formula para Score
@app.get('/score_titulo')
def score_titulo(titulo_de_la_filmacion):
    pelicula = df[df["title"] == titulo_de_la_filmacion]
    
    if pelicula.empty:
        return "No se encontró la filmación en el dataset."
    
    titulo = pelicula["title"].iloc[0]
    ano_estreno = pelicula["release_year"].iloc[0]
    score = pelicula["popularity"].iloc[0]
    
    mensaje = f"La película {titulo} fue estrenada en el año {ano_estreno} con un score/popularidad de {score}."
    
    return mensaje

In [ ]:
# Convertir columna a tipo datetime
df['release_year'] = pd.to_datetime(df['release_year'])

# Extraer el mes de la columna 'release_date'
df['release_month'] = df['release_year'].dt.month

In [ ]:
#Cambio los numeros a su mes correspondiente
def convertir_mes_letras(numero_mes):
    meses = {
        1: "enero",
        2: "febrero",
        3: "marzo",
        4: "abril",
        5: "mayo",
        6: "junio",
        7: "julio",
        8: "agosto",
        9: "septiembre",
        10: "octubre",
        11: "noviembre",
        12: "diciembre"
    }
    
    if numero_mes in meses:
        return meses[numero_mes]
    else:
        return "Mes inválido"

# Supongamos que tienes un DataFrame llamado df con una columna "mes" que contiene números de mes
df["release_month"] = df["release_month"].map(convertir_mes_letras)

In [ ]:
#Formula para la cantidad de filmaciones por MES
@app.get('/cantidad_filmaciones_mes')
def cantidad_filmaciones_mes(mes):
    mes = mes.lower()
    peliculas_mes = df[df["release_month"] == mes]
    cantidad = len(peliculas_mes)
    
    mensaje = f"{cantidad} cantidad de películas fueron estrenadas en el mes {mes.capitalize()}."
    
    return mensaje

In [ ]:
#Sacamos los dias de datetime creando la clumna de solo dias
df['release_day'] = df['release_year'].dt.day

In [ ]:
#Se obtiene el nombre de cada dia en formato texto
df["release_year"] = pd.to_datetime(df["release_year"])  

def obtener_nombre_dia(fecha):
    nombre_dia = fecha.strftime("%A")
    return nombre_dia

df["nombre_dia"] = df["release_year"].apply(obtener_nombre_dia)

In [ ]:
#Pasamos los dias de Ingles a Español
diccionario_dias = {
    "Monday": "Lunes",
    "Tuesday": "Martes",
    "Wednesday": "Miercoles",
    "Thursday": "Jueves",
    "Friday": "Viernes",
    "Saturday": "Sábado",
    "Sunday": "Domingo"
}

df["nombre_dia_espanol"] = df["nombre_dia"].map(diccionario_dias)

In [ ]:
#Formula para ver cuantas peliculas fueron estrenadas por cierto dia
@app.get('/cantidad_filmaciones_dia/{dia}')
def cantidad_filmaciones_dia(dia):
    dia = dia.lower()
    filmaciones_dia = df[df["dia_espanol"].str.lower() == dia]
    cantidad = len(filmaciones_dia)
    
    mensaje = f"{cantidad} cantidad de películas fueron estrenadas en el día {dia.capitalize()}."
    
    return mensaje

In [ ]:
#Se boran columnas que no sirven para limpiar el df
df.drop('nombre_dia', axis=1, inplace=True)
df.drop('original_language', axis=1, inplace=True)

In [ ]:
#Se cambia el nombre de una nueva columna
df.rename(columns={'nombre_dia_espanol':'dia_espanol'}, inplace=True)

In [ ]:
#Ultimo push para guardar el dataset listo
df.to_csv('Csv_Proyecto_Terminado.csv', index=False)